In [76]:
import os

import numpy as np
import scipy.misc
from sklearn.cross_validation import *
from sklearn.metrics import *
from sklearn.preprocessing import *
import tensorflow as tf
from PIL import Image

In [77]:
RED = 0
GREEN = 1
BLUE = 2

In [78]:
def determine_color_from_filename(filename):
    if 'blue' in filename.lower():
        return BLUE
    if 'green' in filename.lower():
        return GREEN
    return RED

In [79]:
def scale_unit_feature(X):
    unit_scaler = MinMaxScaler((0, 1))
    original_size = X.shape
    X = np.reshape(X, (X.shape[0], np.prod(X.shape[1:])))
    X = unit_scaler.fit_transform(X)
    X = np.reshape(X, original_size)
    return X

In [80]:
def convert_to_one_hot(y):
    return OneHotEncoder(3, sparse=False).fit_transform(y)

In [81]:
raw_X = []
raw_y = []
for folder_path, folders, filenames in os.walk('data/input'):
    for filename in filenames:
        filepath = os.path.join(folder_path, filename)
        image = Image.open(filepath).convert('RGB')
        image = np.array(image)
        raw_X.append(image)
        raw_y.append(determine_color_from_filename(filename))

X = np.array(raw_X, dtype='float')
y = np.array(raw_y, dtype='int32').reshape((-1, 1))

In [82]:
print('X.shape: ', X.shape)
print('y.shape: ', y.shape)

X.shape:  (470, 64, 64, 3)
y.shape:  (470, 1)


In [83]:
# X = scale_unit_feature(X)
# y = convert_to_one_hot(y)

In [84]:
print('X.shape: ', X.shape)
print('y.shape: ', y.shape)

X.shape:  (470, 64, 64, 3)
y.shape:  (470, 1)


Begin Tensorflow Code

In [91]:
feature_columns = [
    tf.contrib.layers.real_valued_column("", dimension=64),
    tf.contrib.layers.real_valued_column("", dimension=64),
    tf.contrib.layers.real_valued_column("", dimension=3),
]

In [97]:
feature_columns = tf.contrib.learn.infer_real_valued_columns_from_input(X[0])

AttributeError: 'module' object has no attribute 'infer_real_valued_columns_from_input'

In [92]:
classifier = tf.contrib.learn.DNNClassifier(feature_columns=feature_columns,
                                            hidden_units=[10],
                                            n_classes=3,
                                            model_dir="lab_model.dat")


In [93]:
classifier.fit(x=X, y=y, steps=100)

InvalidArgumentError: ConcatOp : Dimensions of inputs should match: shape[0] = [1925120,3] vs. shape[1] = [90240,64]
	 [[Node: input_from_feature_columns/concat = Concat[N=2, T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/gpu:0"](input_from_feature_columns/concat/concat_dim, input_from_feature_columns/Reshape, input_from_feature_columns/Reshape_1)]]
	 [[Node: zero_fraction/Mean/_133 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_264_zero_fraction/Mean", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]
Caused by op 'input_from_feature_columns/concat', defined at:
  File "/usr/lib/python3.4/runpy.py", line 170, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.4/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py", line 3